# fast hack which aligns the patients

In [14]:
# get a list of patients for fast testing
patients = [p for p in sorted(df['patient'].unique())]
#patients

In [16]:
# in short: create one generator per patient
# this is a fast hack, it is not very fast
# DataGenerator for SAX, changed from PhaseRegression Generator
# logging.getLogger().setLevel(logging.INFO)
from src.data.Generators import DataGenerator
from logging import info
config['SHUFFLE'] = False
config['AUGMENT'] = False
config['AUGMENT_GRID'] = False# make sure no augmentation will be applied to the validation data
config['HIST_MATCHING'] = False
config['BATCHSIZE'] = 1
# by this we can have a different set of parameters for both generators
val_config = config.copy()
export_root = '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp_predictions/k-fold_fold3'
ensure_dir(export_root)

df_fold = df[df['modality']=='test']
#print(df_fold)

# filter a list of filenames by a patient id, this is necessary as the filepath in our df differs from the real filenames
def filter_by_patient_id(p_id, f_names):
    return [elem for elem in f_names if p_id in elem]


#for p in df['patient'].unique(): # show only data on 'unique' patients to sum up folds and slices
for p in sorted(df_fold['patient'].unique()): # for each patient
    info(p) # shows which patient we are at
    files_ = filter_by_patient_id(p, x_val_sax)
    masks_ = filter_by_patient_id(p, y_val_sax)
    info(len(files_)) # shows amount of slices for each patient
    #print(files_[0]) # files is a list with the name of all patient data
    # collect all files for this patient
    # split in ED and ES
    ed_f = files_[:len(files_)//2]
    es_f = files_[len(files_)//2:]
    ed_m = masks_[:len(masks_)//2]
    es_m = masks_[len(masks_)//2:]
    f_ = [ed_f, es_f]
    m_ = [ed_m, es_m]
    phases = ['ED', 'ES']
    assert(len(ed_m)==len(ed_f)), 'number of images and masks should be the same, something went wrong'
    info('length of ed_f ' + str(len(ed_f)))
    info('length of es_f ' + str(len(es_f)))
    #print('this is ed_f ' + ed_f)
    #print('this is es_f ' + es_f)
    for p_ in range(2):
        phase_cmr_files = f_[p_] 
        phase_mask_files = m_[p_]
        current_phase = phases[p_]
        info('patient: {}, phase: {}, files: {}'.format(p, current_phase, len(phase_cmr_files)))
        
        validation_generator = DataGenerator(phase_cmr_files, phase_mask_files, config=val_config)

        # get cmr mask
        gts = np.stack([np.squeeze(y) for x, y in validation_generator])
        logging.info('groundtruth shape' + str(gts.shape))
        #get cmr image
        gts_cmr = np.stack([np.squeeze(x) for x, y in validation_generator])
        logging.info('original cmr shape' + str(gts_cmr.shape))

        # predict on the validation generator
        preds = model.predict(validation_generator)
        logging.info(preds.shape)

        # upper = 1, lower == 2
        # transform to int representation (one-hot)
        gts_flat = np.zeros((gts.shape[:-1]))
        gts_flat[gts[...,0]>0.5] = 1
        gts_flat[gts[...,1]>0.5] = 2

        preds_flat = np.zeros((gts.shape[:-1]))
        preds_flat[preds[...,0]>0.5] = 1
        preds_flat[preds[...,1]>0.5] = 2
        
        info(gts_flat.shape)
        info(preds_flat.shape)
        info(gts_cmr.shape)
        #print(preds_flat) #NotIncluded
        gt_sitks = sitk.GetImageFromArray(gts_flat.astype(np.uint8))
        pred_sitks = sitk.GetImageFromArray(preds_flat.astype(np.uint8))
        gt_cmr_sitks = sitk.GetImageFromArray(np.stack(gts_cmr, axis = 0))
        
        #sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_cmr[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_new_temp_cmr.nrrd')
        
        # please write here the pred_sitk and gt_sitk and gt_cmr_sitk to disk
        sitk.WriteImage(gt_sitks, os.path.join(export_root, '{}_{}_gt.nrrd'.format(p, current_phase)))
        sitk.WriteImage(pred_sitks, os.path.join(export_root, '{}_{}_pred.nrrd'.format(p, current_phase)))
        sitk.WriteImage(gt_cmr_sitks, os.path.join(export_root, '{}_{}_cmr.nrrd'.format(p, current_phase)))
        #sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_cmr[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_new_temp_cmr.nrrd')

logging.info('done! Check the ' + export_root + 'folder for files')

2021-07-06 16:36:24,317 INFO patient007
2021-07-06 16:36:24,318 INFO 20
2021-07-06 16:36:24,318 INFO length of ed_f 10
2021-07-06 16:36:24,318 INFO length of es_f 10
2021-07-06 16:36:24,319 INFO patient: patient007, phase: ED, files: 10
2021-07-06 16:36:24,319 INFO Create DataGenerator
2021-07-06 16:36:24,320 INFO Datagenerator created with: 
 shape: [128, 128]
 spacing: [1.8, 1.8]
 batchsize: 1
 Scaler: MinMax
 Images: 10 
 Augment: False 
 Thread workers: 8
2021-07-06 16:36:24,320 INFO No augmentation
2021-07-06 16:36:24,372 INFO groundtruth shape(10, 128, 128, 2)
2021-07-06 16:36:24,407 INFO original cmr shape(10, 128, 128)
2021-07-06 16:36:26,070 INFO (10, 128, 128, 2)
2021-07-06 16:36:26,072 INFO (10, 128, 128)
2021-07-06 16:36:26,072 INFO (10, 128, 128)
2021-07-06 16:36:26,072 INFO (10, 128, 128)
2021-07-06 16:36:26,075 INFO patient: patient007, phase: ES, files: 10
2021-07-06 16:36:26,075 INFO Create DataGenerator
2021-07-06 16:36:26,076 INFO Datagenerator created with: 
 shape:

# Experimental Combiner

In [8]:

# # write 3d_nrrd file to disk
# #for p in df['patient'].unique(): # show only data on 'unique' patients to sum up folds and slices
# filecounter = 0
# for p in sorted(df['patient'].unique()):
#     print(p)
#     files_ = df[df['patient'] ==p]['x_path'].values
#     print(len(files_)) # shows amount of slices for each patient
#     #print(files_[0]) # files is a list with the name of all patient data
#     ed_f = files_[:len(files_)//2]
#     es_f = files_[len(files_)//2:]
#     print('length of ed_f ' + str(len(ed_f)))
#     print('length of es_f ' + str(len(es_f)))
#     #print('this is ed_f ' + ed_f)
#     #print('this is es_f ' + es_f)
#     print('filecounter starts at at: ' + str(filecounter))
#     limit = filecounter + (len(files_)//2)
#     print('limit starts at: ' + str(limit))
    
    
#     print('WRITING from :' + str(filecounter) + ' to ' + str(limit))
#     #sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_flat[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_new_temp_gt.nrrd')
#     #sitk.WriteImage(sitk.GetImageFromArray(np.stack(preds_flat[filecounter:len(files_)//2], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/ preds{}_ed.nrrd'.format(str(p))
#     sitk.WriteImage(sitk.GetImageFromArray(np.stack(preds_flat[filecounter:limit], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/preds_{}_ed.nrrd'.format(str(p)))
#     #sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_cmr[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_new_temp_cmr.nrrd')
    
#     #sitk.WriteImage(sitk.GetImageFromArray(np.stack(preds_flat[11:20], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/checkpred.nrrd')
    
    
#     filecounter = filecounter + (len(files_)//2)
#     limit = filecounter + (len(files_)//2)
#     print('first round done. filecounter at: ' + str(filecounter) + ', limit at ' + str(limit))
    
    
#     print('WRITING from :' + str(filecounter+1) + ' to ' + str(limit))
#     sitk.WriteImage(sitk.GetImageFromArray(np.stack(preds_flat[filecounter:limit], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/preds_{}_es.nrrd'.format(str(p)))
#     #sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_flat[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_new_temp_gt.nrrd')
#     #sitk.WriteImage(sitk.GetImageFromArray(np.stack(preds_flat[filecounter:len(files_)//2], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/preds{}_ed.nrrd'.format(p))
#     #sitk.WriteImage(sitk.GetImageFromArray(np.stack(gts_cmr[:10], axis=0)), '/mnt/ssd/git/wft21_septum_landmark_detection/data/temp/3d_new_temp_cmr.nrrd')
    
#     filecounter = filecounter + (len(files_)//2)
#     limit = 0
#     print('patient done. filecounter exits at: ' + str(filecounter) + ', limit will be reinitialized in next pass.')
    
#     print('Done')


## Create Images from Arrays

One-hot encoding of annotations. Channel 1 is transformed to pixels with the value of 1, Channel 2 is transformed to pixels with the value of 2. This is the same format to annotations created as .nrrd-files with the MITK-Toolkit. 

Both the predictions and groundtruths are recreated using this method. This is the simplest way for us to scale the predictions to the same size of the groundtruth images, without needing implement undoing of the generator steps and dealing with differenzt sizing or spacing issues. 

Using the sitk toolkit a total of three images are created per training file. One representing the file without any annotations, one with our groundtruth and one with the prediction of the model.

The above created numpy stack is used for this

In [24]:
gts = stack[0]
preds = stack[1]


# upper = 1, lower == 2
# transform to int representation (one-hot)
gts_flat = np.zeros((gts.shape[:-1]))
gts_flat[gts[...,0]>0.5] = 1
gts_flat[gts[...,1]>0.5] = 2

preds_flat = np.zeros((gts.shape[:-1]))
preds_flat[preds[...,0]>0.5] = 1
preds_flat[preds[...,1]>0.5] = 2
print(preds_flat.shape)
#print(preds_flat) #NotIncluded
gt_sitks = [sitk.GetImageFromArray(elem) for elem in gts_flat]
pred_sitks = [sitk.GetImageFromArray(elem) for elem in preds_flat]

(476, 128, 128)


In [25]:
# save preds_flat to understand what they look like

pred_flat_path = os.path.join(config['EXP_PATH'], 'predictions_flat'+ str(datetime.datetime.now().strftime("%Y-%m-%d_%H_%M")))
ensure_dir(pred_flat_path)
pred_flat_filename = os.path.join(pred_flat_path, 'gtpred_fold{}.npy'.format(config['FOLD'])) #names file according to fold, in our case only fold0


np.save(pred_flat_filename, np.stack(preds_flat[:10], axis=0)) # saves a numpy stack of first 10 files only???

logging.info('saved as: \n{} \ndone!'.format(pred_flat_filename))


2021-07-06 11:24:43,928 INFO saved as: 
exp/temp/spacing_1_8/2021-05-26_13_40/predictions_flat2021-07-06_11_24/gtpred_fold0.npy 
done!
